## 1. Outline

1. Basic motivation for hyperparater tuning
2. Integrating google colab with local runtime 

**Run this code**

jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0 --no-browser

3. Installing MLFlow and basic use case
4. Logging parameters and metrics with MLFlow
5. Logging image artifacts with MLFlow
6. Logging PyTorch models with MLFlow and loading them

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim

import mlflow
import mlflow.pytorch
from tqdm import tqdm_notebook

## 2. Params

In [0]:
class Params(object):
  def __init__(self, batch_size, epochs, seed, log_interval):
    self.batch_size = batch_size
    self.epochs = epochs
    self.seed = seed
    self.log_interval = log_interval
    
args = Params(256, 4, 0, 20)

## 3. Dataset

In [21]:
os.getcwd()

'/Users/apple/Dropbox/Data science/Courses/Deep Learning - Padh_ai/Hyperparameter tuining'

In [5]:
folder = r'/Dropbox/Data science/Courses/Deep Learning - Padh_ai/Hyperparameter tuining'
os.chdir(folder)

FileNotFoundError: ignored

In [0]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_set = datasets.MNIST('data',train=True,download=True, transform = transform)
test_set = datasets.MNIST('data', train=True, download=True, transform = transform)

train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.batch_size, shuffle=False)

## 4. Model

In [0]:
class network(nn.Module):
  def __init__(self, nh=32):
    super(network, self).__init__()
    self.classifier = nn.Sequential(
        nn.Linear(784, nh),
        nn.ReLU(),
        nn.Linear(nh,10)
    )
    
  def forward(self,X):
    x = X.view(X.size(0),-1)
    x = self.classifier(x)
    return x
    
    
model = network()
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(), lr=0.01)

In [0]:
# train method to run over the data once

def train():
  # book keeping the metrics
  
  loss_arr = []
  accuracy = []
  model.train()
  for batch_id, data  in tqdm_notebook(enumerate(train_loader),unit='batch',total=len(train_loader)//args.batch_size):
    images, labels = data
    output = model(images)
    loss = loss_fn(output,labels)
    loss.backward()
    opt.step()
    opt.zero_grad()

    if batch_id % args.log_interval == 0:
      mlflow.log_metric('train_loss',loss.data.item()/len(images))  # compute the loss per image
  
# method for evaluating the data 

# put the model to eval mode
# compute the loss
# compute the accuracy

def test():
  model.eval()
  correct = 0
  test_loss = 0
  with torch.no_grad():
    for i, data in tqdm_notebook(enumerate(test_loader),unit='batch',total=len(test_loader)//args.batch_size):

      images, labels = data
      test_pred = model(images)
      test_loss += loss_fn(test_pred,labels).data.item()
      correct += torch.sum(torch.argmax(test_pred,dim=1) == labels).item()

    test_loss /= len(images)  # computing the average loss per batch
    accuracy = correct/len(test_loader.dataset)
    mlflow.log_metric('test_loss',test_loss)
    mlflow.log_metric('accuracy',accuracy)
    

In [38]:
for lr in [0.001,0.01,0.1]:
  print('Learning rate:',lr)
  
  model = network()
  loss_fn = nn.CrossEntropyLoss()
  opt = optim.SGD(model.parameters(), lr=lr)
  
  with mlflow.start_run(nested=True) as run:
    for key, value in vars(args).items():
      mlflow.log_param(key,value)
    mlflow.log_param('learning_rate',lr)
    
    train()
    test()

Learning rate: 0.001


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

Learning rate: 0.01


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

Learning rate: 0.1


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))